In [1]:
!pip install -Uqq langchain langchain-openai langchain-experimental

In [2]:
import os
import requests
import json

from langchain_core.tools import tool
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4-turbo"

URL="http://zion.local:8000"

In [4]:
@tool
def make_api_call(method, endpoint, params):
    """ Make an API call to an endpoint given the HTTP method, endpoint, and parameters to send as a dict """
    url = endpoint

    if method == "GET":
        res = requests.get(url, params)
    elif method == "POST":
        res = requests.post(url, data=json.dumps(params))
    else:
        raise Exception(f"Unknown method: {method}")

    if res.status_code != 200:
        return json.dumps({
            "error_code": res.status_code,
            "error_msg": res.text
        })
    else:
        return res.json()

In [5]:
tools = [
    make_api_call,
]

In [6]:
system_prompt = f"""
    You are playing a detective game like 'where in the world is Carmen San Diego'. You will have to make a lot of 
    decisions so look at the clues carefully. The server is hosted at {URL}. The API specification is at the 
    endpoint /openapi.json. 

    To play the game, do the following:
    1. Make an API call to get the schema and understand it
    2. Make an API call to start a new game
    3. Make an API call to get the clues for the next place to travel to
    4. Make an API call to get the next destinations
    5. Based on the clues pick the destination to go to next
    6. Make an API call to travel to the destination
    7. Keep doing steps 2 - 6 until you get a clue that says that you have arrested the suspect.

    Always use the full url for making the API calls. If you get an error, retry the step.
"""
prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      ("placeholder", "{chat_history}"),
      ("human", "{input}"),
      ("placeholder", "{agent_scratchpad}"),
    ]
)

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=50)

res = agent_executor.invoke(
    {
        "input": "Play the game using the tools available"
    }
)

res["output"]



> Entering new AgentExecutor chain...

Invoking: `make_api_call` with `{'method': 'GET', 'endpoint': 'http://zion.local:8000/openapi.json', 'params': {}}`


{'openapi': '3.1.0', 'info': {'title': 'FastAPI', 'version': '0.1.0'}, 'paths': {'/new_game': {'post': {'summary': 'Starts a new game', 'operationId': 'new_game_new_game_post', 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}}}}, '/get_destinations': {'get': {'summary': 'Gets the next set of destinations', 'description': 'Gets the possible destinations to travel to next to solve the case specified by case_id', 'operationId': 'get_destinations_get_destinations_get', 'parameters': [{'name': 'case_id', 'in': 'query', 'required': True, 'schema': {'type': 'string', 'title': 'Case Id'}}], 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}, '422': {'description': 'Validation Error', 'content': {'application/json': {'sche

'Congratulations! You have successfully arrested the suspect, Diamond Dave, in Tokyo. The game is now complete. Well done on solving the case! 🎉🕵️\u200d♂️'